In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cProfile, pstats, sys
import cv2
import sys

from modules.path_planning.astar import AStar
from modules.path_planning.BFS import BFS
from modules.path_planning.greedyBestFirst import greedyBestFirst
from modules.path_planning.bidirectDijkstras import BiDirectionalDijkstra
from modules.path_planning.RRT_star import RRT_Star
from modules.path_planning.INFORMED_RRT import Informed_RRT_star
from modules.path_planning.RRT import RRTtree
from modules.path_planning.RRA import RRASEARCHTREE
from modules.path_planning.BIT import BIT, ELLIPSE
from modules.path_planning.Theta import Theta
import time
import random
import math
import os
import copy
from os.path import isfile,join

#Global variables to store the mouse position coordinates from the image
startX, startY, endX, endY = 0,0,0,0
selectStart = True
selectEnd = True
img = None
img1 = None
path = None
path_runner = None
cv2.namedWindow('image')

def select_point(event,x,y,flags,param):
    """
    Grabs points selected from the cv2 image shown on screen
    """
    global startX,startY,endX,endY,img,selectStart, img1, selectEnd
    

        

    if event == cv2.EVENT_LBUTTONDBLCLK: # captures left button double-click
        ix,iy = x,y

        #Check to see if the user selected a point inside a road
        if not np.array_equal(img[iy][ix],[255,255,255]):
            print("Please select a valid point inside the road!")
            return
            
        #Set the start coordinate if selectStart is still true
        if selectStart:
            img = cv2.circle(img,(ix,iy),radius=5,color=(255,0,0),thickness=-1)
            selectStart = False
            startX,startY = ix,iy
            cv2.imshow('image',img)
            cv2.waitKey(1)
        #Set the endPoint and begin path planning computation
        elif selectEnd:
            selectEnd = False
            img = cv2.circle(img,(ix,iy),radius=5,color=(0,0,255),thickness=-1)
            endX,endY = ix,iy
            cv2.imshow('image',img)
            cv2.waitKey(1)

            #Start the path planning algoirthm
            runPathPlanning()

            #Display the results to the user
            #Resize the image if needed
            #img = cv2.resize(img,(1024,1024),interpolation=cv2.INTER_AREA)
            cv2.imshow('image',img)
         #   selectStart = True


def grabRandomImage():
    #Returns an image path to use for the path processing

    imgFolder = "C:/Users/charles/Downloads/sample_predictions2/predictions"
    groundTruthFolder = "C:/Users/charles/Downloads/sample_predictions2/ground_truth"
    imgs = [f for f in os.listdir(imgFolder) if isfile(join(imgFolder,f))]

    #return join(imgFolder,random.choice(imgs))
    return join(imgFolder, imgs[4]), join(groundTruthFolder, imgs[4])

def runPathPlanning(algoName="RRA*"):
    start = time.time()
    startPos = (startX,startY)
    endPos = (endX,endY)
    global path
    global path_runner

    if algoName == "astar":

        #Run the pathfinding algorithm
        path_runner = AStar(startPos,endPos,copy.deepcopy(img),"image")
        path = path_runner.run()
        print("Path length:",len(path))
    
    elif algoName == "Theta":
        path_runner = Theta(startPos,endPos, copy.deepcopy(img), "image")
        pr = cProfile.Profile()
        pr.enable()
        path = path_runner.run()
        pr.disable()
        ps = pstats.Stats(pr, stream=sys.stdout)
        ps.print_stats()
        
    elif algoName == "BIT":
        path_runner = BIT(startPos, endPos, 5000, copy.deepcopy(img))
        path = path_runner.run()
    elif algoName == "bfs":
        path_runner = BFS(startPos,endPos,copy.deepcopy(img),"image")
        path = path_runner.run()
    
    elif algoName == "greedyBestFirst":
        path_runner = greedyBestFirst(startPos,endPos,copy.deepcopy(img),"image")
        path = path_runner.run()
    elif algoName == "bidirectDijkstras":
        path_runner = BiDirectionalDijkstra(startPos,endPos,copy.deepcopy(img),"image")
        path = path_runner.run()
    elif algoName == "RRT":
        path_runner = RRTtree(startPos, endPos, 4, 10000, copy.deepcopy(img), "image")
        path = path_runner.run()
    
    elif algoName == "RRT_Star":
        path_runner = RRT_Star(startPos, endPos, 10, 10000, 50, copy.deepcopy(img))
        pr = cProfile.Profile()
        pr.enable()
        path = path_runner.run()
        pr.disable()
        ps = pstats.Stats(pr, stream=sys.stdout)
        ps.print_stats()   
    
    elif algoName == "INFORMED RRT*":
        path_runner = Informed_RRT_star(startPos, endPos, 2000, 10, 30, 150, copy.deepcopy(img))
        path = path_runner.run()
    
    elif algoName == "RRA*":
        path_runner = RRASEARCHTREE(startPos, endPos, copy.deepcopy(img), "image", endPos)
        pr = cProfile.Profile()
        pr.enable()
        path = path_runner.pathCalculation()
        pr.disable()
        ps = pstats.Stats(pr, stream=sys.stdout)
        ps.print_stats()   

        
    else:
        print("No valid algorithm found for the algorithm name given...exiting")
        exit()
    #else algoName == "LPA":
    #Future path finding algorithms go here..each should return a list of coordinates
    
    end = time.time()
    duration = end - start
    #cv2.line(img, start point, end point, color=(0, 255, 0), 3)
    #Plot the path found onto the image
    solution_quality = 0.0
    map_path = []
    first_point = path[0]
    second_point = None

    for pos in path:
        map_path.append(pos)
  #      cv2.rectangle(img,pos,(pos[0]+2,pos[1]+2),color=(0,0,255),thickness=-1)
        if pos != first_point:
            second_point = pos
            cv2.line(img, first_point, second_point, color=(0, 255, 0), thickness=5)
            first_point = pos
        if pos != path[0]:
            solution_quality += math.hypot((map_path[-1][0] - map_path[-2][0]),(map_path[-1][1] - map_path[-2][1]))

    print('Quality/Cost of solution: ', solution_quality)
    print("Time took: ",duration)
    simulate()


    
    
    
    
def simulate(circle_radius=30):
    global img, img1
    global path
    global path_runner
    global startX, startY, endX, endY
    time.sleep(5)

    i = 0
    currentPosition = path[i] if path else None
    assert(currentPosition is not None)
    changed = False
    while currentPosition is not None:
        pl = list(set(ELLIPSE(circle_radius,circle_radius,int(math.pi*circle_radius**2),currentPosition,currentPosition).ret_list()))
        for p in pl:
            if p[0] >= len(img[0]) or p[0] < 0 or p[1] >= len(img) or p[1] < 0:
                continue
            if np.array_equal(img[p[1]][p[0]], img1[p[1]][p[0]]) is False and (np.array_equal(img[p[1]][p[0]], [0,255,0]) and np.array_equal(img1[p[1]][p[0]],[0,0,0])) or (np.array_equal(img[p[1]][p[0]], [0,0,0]) and np.array_equal(img1[p[1]][p[0]],[255,255,255])) or (np.array_equal(img[p[1]][p[0]], [255,255,255]) and np.array_equal(img1[p[1]][p[0]],[0,0,0])):
                img[p[1]][p[0]] = img1[p[1]][p[0]]
        i +=1
        cv2.imshow('image',img)
        if i >= len(path):
            break
        currentPosition = path[i]

    
    
    
    
    
    
#use right bumper of mouse. 
def replan():
    #Store dynamic bool, algo_path_State, algoname as params. 
    global img
    global path
    global img1
    global path_runner
    global startX,startY,endX,endY

    glass = random.sample(path, 5)
    print('Replanning from start position:', (startX, startY), 'to goal position:', (endX, endY))
    for pos in path:
        if np.array_equal(img[pos[1]][pos[0]], [0, 255, 0]):
            img[pos[1]][pos[0]] = [255, 255, 255]

    for a in glass:
        if a == (startX, startY) or a == (endX, endY):
            continue
        img[a[1]][a[0]] = [0, 0, 0]
        print('Added obstactle at', a)

                    
    img = cv2.circle(img,(endX,endY),radius=5,color=(0,0,255),thickness=-1)
    img = cv2.circle(img,(startX,startY),radius=5,color=(255,0,0),thickness=-1)
    path.clear()

                

    start = time.time()
    path_runner = RRASEARCHTREE((startX, startY), (endX, endY), copy.deepcopy(img), "image", (endX, endY))
    path_runner.img2 = img
    path = path_runner.pathCalculation()    

    print(path)

    end = time.time()
    duration = end - start
    #cv2.line(img, start point, end point, color=(0, 255, 0), 3)
    #Plot the path found onto the image
    solution_quality = 0.0
    map_path = []
    first_point = path[0]
    second_point = None

    for pos in path:
        map_path.append(pos)
  #      cv2.rectangle(img,pos,(pos[0]+2,pos[1]+2),color=(0,0,255),thickness=-1)
        if pos != first_point:
            second_point = pos
            cv2.line(img, first_point, second_point, color=(0, 255, 0), thickness=1)
            first_point = pos
        if pos != path[0]:
            solution_quality += math.hypot((map_path[-1][0] - map_path[-2][0]),(map_path[-1][1] - map_path[-2][1]))
        #img[pos[1]][pos[0]] = [0,255,0]

    print('Quality/Cost of solution: ', solution_quality)
    print("Time took: ",duration)

    cv2.imshow('image', img)




        
    
    
    

def main():
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"

    global img 
    global img1
    imgPath, imgPathgroundTruth = grabRandomImage()

    #Load the image
    img = cv2.imread(imgPath)
    img1 = cv2.imread(imgPathgroundTruth)

    #Resize the image if desired (Speeds up computation)
    #img = cv2.resize(img,(512,512),interpolation=cv2.INTER_AREA)
    #Convert the image to GrayScale
    #img = cv2.cvtColor(cv2.imread('path1.png'),cv2.COLOR_BGR2GRAY)

    #Initialize the image window
    cv2.setMouseCallback('image',select_point)

   # cv2.setMouseCallback('image', lambda *args : None)
    #Display image and wait until the user selects two valid points on the image 
    cv2.imshow('image',img)

    k = cv2.waitKey(0) 
    if k == 27:
        print('replan stage')
        param = [path, RRA_SEARCH, path_cost]
        cv2.setMouseCallback('image', replan)
        replan()
        
        
        
        
  #  cv2.destroyAllWindows()



if __name__ == "__main__":
    main()

Beginning search from: (39, 293) to: (342, 670)
In navigation process
path found.
         72037 function calls in 0.119 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1745    0.000    0.000    0.000    0.000 C:\Users\charles\Downloads\Path-Planning-On-Aerial-Images-main-20220523T022800Z-001\Path-Planning-On-Aerial-Images-main\modules\path_planning\RRA.py:12(__init__)
     1759    0.001    0.000    0.001    0.000 C:\Users\charles\Downloads\Path-Planning-On-Aerial-Images-main-20220523T022800Z-001\Path-Planning-On-Aerial-Images-main\modules\path_planning\RRA.py:53(euclidean_distance_grid)
      755    0.003    0.000    0.003    0.000 C:\Users\charles\Downloads\Path-Planning-On-Aerial-Images-main-20220523T022800Z-001\Path-Planning-On-Aerial-Images-main\modules\path_planning\RRA.py:61(get_neighbors)
     2648    0.080    0.000    0.080    0.000 C:\Users\charles\Downloads\Path-Planning-On-Aerial-Images-main-20220523T02